In [74]:
import pandas as pd
import numpy as np 
import plotly.express as px

In [75]:
def simulate_returns(mean, volatility, num_days):
    """
    Simulate Gaussian distributed returns.
    """
    daily_returns = np.random.normal(mean/num_days, volatility/np.sqrt(num_days), num_days)
    return daily_returns


def simulate_returns_correlation(mean, volatility, num_days, correlation):
    """
    Simulate Gaussian distributed returns, with certin correlations using
    the Cholesky decomposition which takes a correlation matrix and generates
    random variables with the desired correlation structure
    """
    cov_matrix = np.array([[1, correlation], [correlation, 1]])

    # Perform Cholesky decomposition
    L = np.linalg.cholesky(cov_matrix)

    returns = np.random.normal(mean, volatility, (2, num_days))  # Generate two sets of uncorrelated random variables
    # Apply Cholesky decomposition to correlate the random variables
    correlated_returns = np.dot(L, returns)

    return correlated_returns[0, :], correlated_returns[1, :]


def calculate_covariance(returnX, returnY): 
    """
    Calculate Covariance between two securities given their returns
    The reason why we use N-1 in the sample is because we want to penalize the sample and add
    slighly more value to make sure we're not getting a number less than the population variance 
    """
    if len(returnX) != len(returnY):
        raise ValueError("Input datasets must have the same length")
    
    covariance = np.cov(returnX, returnY)[0,1]
    
    return covariance

def get_portfolio_retunrs_volatilty(weightX, weightY, returnX, returnY):
    """
    Create a Portfolio of 2 assets and get assets returns and standard deviation
    """

    returns = sum(weightX*returnX[i] + weightY*returnY[i] for i in range(len(returnX)))
    varX = calculate_covariance(returnX, returnX)
    varY = calculate_covariance(returnY, returnY)
    cov = calculate_covariance(returnX, returnY) 
    variance = ((weightX**2) * varX) + ((weightY**2) * varY) + (2 * weightX * weightY * cov)
    
    return returns.sum() , variance


In [ ]:
##### The variables for portfolio creation are assets returns, volatilities, weights and  correlation 
##### I want to investigate how correlation influnce potfolio returns and volatilities and how can it be 
##### a great hedging instrument. For this we will try 10 different correlations, with 10 different weights,
##### and 10 different compinations of volatitilies, we will then plot correlations, returns and volatilty
##### hoping for a trendy shape 

In [89]:
import random 

random.seed(0)
num_days = 100

weightsX = list(np.random.uniform(0, 1, 3)) #Get random weights
weightsY = [1-i for i in weightsX]

correlations = [-0.99, -0.5, 0, 0.5, 0.99] 

final_list = []

for iteration in range(100):
    
    for correlation in correlations: 

        # Step 1: simulate returns 
        returnX, returnY = simulate_returns_correlation(0, random.random(), num_days, correlation)
        
        # Step 2: Get Portfolio returns for different weights 
        for weightx, weightY in zip(weightsX , weightsY): 
            return_, vol = get_portfolio_retunrs_volatilty(weightx, weightY, returnX, returnY)

            #Step 3: Append all portfolio returns_ and volatilties with their weights 
            final_list.append([correlation, weightx, weightY, return_, vol])

final_data = pd.DataFrame(final_list, columns=['correlation', 'weightx', 'weightY', 'return_', 'vol'] )
final_data

,correlation,weightx,weightY,return_,vol
0,-0.99,0.496248,0.503752,-0.886039,0.003669
1,-0.99,0.841497,0.158503,3.317870,0.397899
2,-0.99,0.742158,0.257842,2.108268,0.202443
3,-0.50,0.496248,0.503752,2.574958,0.120843
4,-0.50,0.841497,0.158503,-10.166359,0.347212
...,...,...,...,...,...
1495,0.50,0.841497,0.158503,0.134188,0.004685
1496,0.50,0.742158,0.257842,0.178525,0.004289
1497,0.99,0.496248,0.503752,0.084917,0.137955
1498,0.99,0.841497,0.158503,-0.223205,0.138492


In [90]:
fig = px.scatter_3d(final_data, x='vol', y='return_', z='correlation')
fig.show()